In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Run on colab only
#!pip install transformers
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-qvfidlbw
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-qvfidlbw
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.1.0.dev0-cp36-none-any.whl size=1448024 sha256=b165ba439017aaea58bd4564b9c4db5d30e79845e8c75996ab134303ba25ed06
  Stored in directory: /tmp/pip-ephem-wheel-cache-9amkggrt/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers


In [3]:
#Run on colab only
!pip install emoji --upgrade

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.6.0)


In [4]:
#Run on colab only
#cd to folder which contain required external files
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/cs410/CourseProject 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/cs410/CourseProject


In [5]:
import tensorflow as tf 
import tensorflow_hub as hub 
from tensorflow.keras import layers
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import BertTokenizer, TFBertForSequenceClassification
import sklearn
import numpy as np 
import pandas as pd 
import json
import re
import random
import math
from TEXT_PREPROCESSING import preprocess_text
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# LOADING DATA
categorized_tweets = pd.read_json('./data/train.jsonl', lines = True)
categorized_tweets.isnull().values.any()
print(categorized_tweets)

# PREPROCESSING DATA
tweets = []
data = list(categorized_tweets["response"])
print(data[0])
for d in data:
    tweets.append(preprocess_text(d))

y = categorized_tweets["label"]
y = np.array(list(map(lambda x: 1 if x=="SARCASM" else 0, y)))
print(tweets[9])

            label  ...                                            context
0         SARCASM  ...  [A minor child deserves privacy and should be ...
1         SARCASM  ...  [@USER @USER Why is he a loser ? He's just a P...
2         SARCASM  ...  [Donald J . Trump is guilty as charged . The e...
3         SARCASM  ...  [Jamie Raskin tanked Doug Collins . Collins lo...
4         SARCASM  ...  [Man ... y ’ all gone “ both sides ” the apoca...
...           ...  ...                                                ...
4995  NOT_SARCASM  ...  [@USER Apologies for the inconvenience you fac...
4996  NOT_SARCASM  ...  [@USER 🤔 idk tho , I think I ’ m #hungry . But...
4997  NOT_SARCASM  ...  [@USER @USER @USER Peace to you , and two coun...
4998  NOT_SARCASM  ...  [Bernie Sanders told Elizabeth Warren in priva...
4999  NOT_SARCASM  ...  [PDP PROTEST BRAINSTORMING SESSION Deji : We n...

[5000 rows x 3 columns]
@USER @USER @USER I don't get this .. obviously you do care or you would've moved right

In [7]:
# TOKENIZING DATA
train_data, val_data, train_lab, val_lab = sklearn.model_selection.train_test_split(tweets, y, test_size=0.2, random_state=34)

##for more details https://huggingface.co/transformers/model_doc/distilbert.html
Bert_method = 1
if Bert_method == 0:
    model_class = TFDistilBertForSequenceClassification
    tok_class = DistilBertTokenizer
    pretrained_model = 'distilbert-base-uncased'
else:    
    model_class = TFBertForSequenceClassification
    tok_class = BertTokenizer
    pretrained_model = 'bert-base-uncased'

tokenizer = tok_class.from_pretrained(pretrained_model) 
train_tok = tokenizer(train_data, truncation=True, padding=True)
val_tok = tokenizer(val_data, truncation=True, padding=True)

In [8]:
type(train_tok)

transformers.tokenization_utils_base.BatchEncoding

In [9]:
#print(train_tok.values)
print(train_tok.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [10]:
d = dict(train_tok)
d.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [11]:
# Create dataset object
train_tok_as_dict = dict(train_tok)
val_tok_as_dict = dict(val_tok)
train_data = tf.data.Dataset.from_tensor_slices((train_tok_as_dict,train_lab))
val_data = tf.data.Dataset.from_tensor_slices((val_tok_as_dict, val_lab))

In [12]:
# Build Model
model = model_class.from_pretrained(pretrained_model, num_labels=2)
opt = tf.keras.optimizers.Adam(learning_rate=.00005,  beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss=model.compute_loss, optimizer= opt, metrics=['accuracy'])
print(model.summary())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# Fit model
model.fit(train_data.batch(32), epochs=2, batch_size=32, validation_data=val_data.batch(32))

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f8896f14e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f8896f14e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


125/125 [==============================] - ETA: 0s - loss: 0.5683 - accuracy: 0.7085

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


125/125 [==============================] - 79s 632ms/step - loss: 0.5683 - accuracy: 0.7085 - val_loss: 0.4676 - val_accuracy: 0.7860
Epoch 2/2
125/125 [==============================] - 79s 634ms/step - loss: 0.4431 - accuracy: 0.8025 - val_loss: 0.4292 - val_accuracy: 0.8100


In [14]:
# Predict using model
uncat_tweets = pd.read_json('./data/test.jsonl', lines = True)
un_tweets = []
uncat_data = list(uncat_tweets["response"])

for d in uncat_data:
    un_tweets.append(preprocess_text(d))
    
tokenized_un_tweets = tokenizer(un_tweets, truncation=True, padding=True, return_tensors='tf')
predictions = tf.nn.softmax(model(tokenized_un_tweets)[0], axis=1).numpy()[:,1]

In [15]:
predictions[0:10]

array([0.3958142 , 0.8792334 , 0.8628402 , 0.6030083 , 0.80222267,
       0.8955111 , 0.0268482 , 0.877433  , 0.8274313 , 0.8739387 ],
      dtype=float32)

In [16]:
# Write predictions to file
with open('answer_x.txt', 'w') as f:
    c = 1
    s_c = 0
    ns_c = 0
    for p in predictions:
        if p >= .5:
            f.write("twitter_" + str(c) + "," + "SARCASM\n")
            c += 1
            s_c += 1
        else:
            f.write("twitter_" + str(c) + "," + "NOT_SARCASM\n")
            c += 1
            ns_c += 1
print("# sarcasm: " + str(s_c))
print("# not sarcasm: " + str(ns_c))


# sarcasm: 996
# not sarcasm: 804


In [17]:
f

<_io.TextIOWrapper name='answer_x.txt' mode='w' encoding='UTF-8'>